In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-h348hvoy
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-h348hvoy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=8095d7556677c802540cccc787abf573b95021efd8a78cf8fbde2ba98deb5bbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-2p1bu02i/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
/**
 * --------------------------------------------------------
 * Universidad del Valle de Guatemala
 * CC3056 - Programación de Microprocesadores
 * --------------------------------------------------------
 * HC-SR04.ipynb
 * Proyecto 3
 * --------------------------------------------------------
 * Javier Prado -21486
 * Angel Perez - 21298
 * Bryan Espana - 21550
 * --------------------------------------------------------
 */
#include <iostream>
#include <sstream>
#include <string>
#include <fstream>
#include <bits/stdc++.h>
//Tamaño de cada vector y block
#define N 500
#define BLOCKSIZE 128 

using namespace std;

//Declarar y asignar memoria de host y device
__global__ void mul(float *distance, float *res){
	//COLOCAR CODIGO PARA CACULAR ID GLOBAL DE TRHEAD PARA ASIGNAR EJECUCION DE TRABAJO
	int myID = threadIdx.x + blockDim.x * blockIdx.x;
  if(myID < N)
  		res[myID] = distance[myID]/0.0343; //obtener el tiempo de cada distancia en microseg
}// mul


int main(int argc, char** argv) {

  //float *h_distance, *h_res;       // declaración se realiza junto a memory allocation
  float *d_distance, *d_res;
    
  //Host memory allocation
  /**
   * Para acomodar un vector de tamaño variable, usamos memoria dinámica (malloc) 
   * Debemos especificar el tipo apuntando al inicio del vector (int *) y el tamaño
   * de cada elemento N del vector (N*size)
   */
  int numElements = 1;  
   size_t size = numElements * sizeof(float);                                                   //size de arreglo en Bytes

  //Declaración & Host memory allocation 
  int* h_distance = (int *)malloc(N*size);
  int* h_res = (int *)malloc(N*size);
  
  //Device memory allocation
  cudaMalloc((void **) &d_distance, N*size);
  cudaMalloc((void **) &d_res, N*size);






    string linea;                                                                                //guarda 1 registro/fila del csv a la vez
    string distancia;    //columnas del txt
                                                                      //cantidad de registros/filas en el csv
    float temp;                                                                                  //variable auxiliar para conversión de str a float
    float *h_distancia = (float *)malloc(size);                                                   //memory allocation del arreglo en el host
    int i;                                                                                       //contador auxiliar
    
    ifstream archivo ("Datos1.txt");          //abrir archivo
    if (archivo.fail()) {                              //error cuando el archivo no puede abrir
        cerr << "Unable to open file" << endl;         
        return 1;
    }
    
    getline(archivo,linea);                            //omitir encabezado del txt (nombres de columnas)
    i = 0;
    while (getline(archivo,linea)) {                   //mientras existan lineas de registro dentro del txt
      stringstream stream(linea);
                                                       // Extraer todos los elementos del registro/fila en variables separadas
      getline(stream, distancia);
      try{
        h_distance[i] = stof(distancia);
      }catch(...){
        h_distance[i] = 0;
      }
 ////Inicialización  de los valores de vectores
      h_distance[i] = temp;
      cout << "Distancia: " << temp << endl;            //imprimir valor
      //temp = stof(distancia);                           //convertir de string a float dato Distancia
      

     
      }
      archivo.close();                                   //cerrar archivo

  //TRASLADO DE DATOS A OPERAR DESDE MEMORIA DE HOST A MEMORIA DE DEVICE
  cudaMemcpy(d_distance, h_distance, N*size, cudaMemcpyHostToDevice);
  
  
  //CALCULO CORRECTO DEL NUMERO DE HILOS POR BLOQUE
  int threadsPerBlock = BLOCKSIZE;
  int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
  
  //LLAMADA DEL KERNEL DESDE EL HOST/CPU
  mul<<<blocksPerGrid, threadsPerBlock>>>(d_distance, d_res);
  /**
   * La suma de hilos de cada bloque debe ser suficiente
   * para cubrir el total del rango de datos
   */
	

  // TRASLADO DE RESULTADOS DEL DEVICE A MEMORIA DEL HOST
  cudaMemcpy(h_res, d_res, N*size, cudaMemcpyDeviceToHost);

  //IMPRESIÓN DE RESULTADOS
  printf("Vector Distancia:\n");
  for (int i=0; i<N; i++)
  {
	  printf("%d ", h_distance[i]);
  }
  printf("\n");
  
  printf("Vector Tiempo:\n");
  for (int i=0; i<N; i++)
  {
	  printf("%d ", h_res[i]);
  }
  printf("\n");

   /**
   * Liberar memoria dinámica en el host y device
   */
  delete [] h_distance;
  delete [] h_res;
  
  cudaFree(d_distance);
  cudaFree(d_res);


    return 0; 

 }

Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Distancia: 4.56753e-41
Vector Distancia:
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
%%cu
#include <stdio.h>

#include <cuda_runtime.h>

#include <fstream>

#include <string>

#include <iostream>

#include <vector>

__global__ void get_time(float * D, float * T, int N) {
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  if (i < N) {      
    T[i] = D[i] / 0.0343;
  }
}

std::vector < float > get_txt_data(std::string path) {
  std::vector < float > data;

  std::ifstream file(path);
  std::string str;

  int first = 1;
  while (std::getline(file, str)) {
    if (first == 0) {
      data.push_back(std::stof(str));
    }
    first = 0;
  }
  return data;
}

int main() {
  std::vector < float > read_data = get_txt_data("Datos1.txt");

  int numElements = read_data.size();
  size_t size = numElements * sizeof(float);

  float * h_D = (float * ) malloc(size);
  float * h_T = (float * ) malloc(size);

  for (int i = 0; i < numElements; i++) {
    h_D[i] = read_data[i];
  }

  float * d_D = NULL;
  cudaMalloc((void **)&d_D, size);   
  float * d_T = NULL
  ;cudaMalloc((void **)&d_T, size);   

  cudaMemcpy(d_D, h_D, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_T, h_T, size, cudaMemcpyHostToDevice);

  int threadsPerBlock = 128; 
  int blocksPerGrid = (numElements + threadsPerBlock - 1) / threadsPerBlock;

  get_time<<<blocksPerGrid, threadsPerBlock>>>(d_D, d_T, numElements);
  
  cudaMemcpy(h_D, d_D, size, cudaMemcpyDeviceToHost);
  cudaMemcpy(h_T, d_T, size, cudaMemcpyDeviceToHost);


  printf("RESULTADOS:\n");
  for (int i = 0; i < numElements; i++) {
    printf("(Distance,Time)[%i]= (%.3f, %.5f)\n", i, h_D[i], h_T[i]);
  }
  
  cudaFree(d_D);
  cudaFree(d_T);

  free(h_D);
  free(h_T);

  return 0;
}

RESULTADOS:
(Distance,Time)[0]= (4.010, 116.90963)
(Distance,Time)[1]= (3.280, 95.62682)
(Distance,Time)[2]= (3.050, 88.92128)
(Distance,Time)[3]= (3.040, 88.62974)
(Distance,Time)[4]= (2.740, 79.88338)
(Distance,Time)[5]= (2.920, 85.13120)
(Distance,Time)[6]= (2.740, 79.88338)
(Distance,Time)[7]= (2.980, 86.88047)
(Distance,Time)[8]= (2.440, 71.13703)
(Distance,Time)[9]= (2.760, 80.46647)
(Distance,Time)[10]= (3.170, 92.41983)
(Distance,Time)[11]= (4.180, 121.86588)
(Distance,Time)[12]= (4.850, 141.39941)
(Distance,Time)[13]= (5.810, 169.38776)
(Distance,Time)[14]= (6.590, 192.12828)
(Distance,Time)[15]= (7.200, 209.91254)
(Distance,Time)[16]= (8.010, 233.52771)
(Distance,Time)[17]= (9.110, 265.59766)
(Distance,Time)[18]= (9.760, 284.54813)
(Distance,Time)[19]= (11.040, 321.86588)

